## Prediction Problem Code Template

- Please only put down the best setup, which means the preprocessing, predictor selection (if applicable), tuning and training, and prediction. The best setup means the best performance on Kaggle Public Leaderboard.
- **Keep in mind that your code needs to run for credit.**
- Choose the problem type and the model type below, before your code. You will use this template for all parts of both regression and classification problems.

**Problem:** Classification

**Model:** Ensemble

**Group Members:** Ben Maisel

**Performance on Kaggle:** Put the RMSE/accuracy the setup achieved here. **We will run your code and it needs to match that performance for credit!**

### Libraries

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.ensemble import VotingRegressor, StackingRegressor, VotingClassifier, StackingClassifier

### Data and Preprocessing

In [7]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')
train.head()
# train.columns
h_id = test.host_id
test.head()

id     host_id  host_since host_location  \
0  1543972437713169913  1521551465  2020-12-16           NaN   
1  1710552057351883447   429737865  2016-12-16  New York, NY   
2             97075525   995041819  2019-03-14           NaN   
3             83734823    97552339  2014-12-02   Chicago, IL   
4             56722823  1029857633  2019-04-22           NaN   

  host_response_time host_response_rate host_acceptance_rate  \
0       within a day                88%                  80%   
1     within an hour               100%                  97%   
2     within an hour                99%                  99%   
3     within an hour               100%                 100%   
4       within a day               100%                  65%   

    host_neighbourhood  host_listings_count  host_total_listings_count  ...  \
0       Near West Side                   44                         44  ...   
1            Cambridge                 4820                       6438  ...   
2  West Loop/Greektown                   71                         88  ...   
3        Streeterville                   14                         47  ...   
4          River North                   20                         22  ...   

  review_scores_checkin review_scores_communication review_scores_location  \
0                   NaN                         NaN                    NaN   
1                   NaN                         NaN                    NaN   
2                  4.82                        4.91                   4.91   
3                  4.67                        4.83                   5.00   
4                  4.67                        4.67                   5.00   

  review_scores_value  instant_bookable  calculated_host_listings_count  \
0                 NaN                 f                              44   
1                 NaN                 f                             643   
2                4.64                 t                              51   
3                4.67                 f                              14   
4                4.67                 f                              19   

  calculated_host_listings_count_entire_homes  \
0                                           2   
1                                         643   
2                                          51   
3                                          14   
4                                          19   

  calculated_host_listings_count_private_rooms  \
0                                           42   
1                                            0   
2                                            0   
3                                            0   
4                                            0   

   calculated_host_listings_count_shared_rooms reviews_per_month  
0                                            0               NaN  
1                                            0               NaN  
2                                            0              1.61  
3                                            0              0.19  
4                                            0              0.16  

[5 rows x 52 columns]

In [8]:
pd.set_option('display.max_columns', None)

# train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage


# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)


train_categories = set(train['host_location'].unique())
test['host_location'] = test['host_location'].apply(lambda x: x if x in train_categories else np.nan)
test['host_location'].fillna(train['host_location'].mode()[0], inplace=True)

train['host_verifications_count'] = train['host_verifications'].apply(lambda x: len(x.split(',')))
test['host_verifications_count'] = test['host_verifications'].apply(lambda x: len(x.split(',')))

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)




train.head()

id  host_response_rate  host_acceptance_rate  \
0            107745711                96.0                  95.0   
1             78539203               100.0                  97.0   
2  1112961466060884803               100.0                 100.0   
3            104794437               100.0                  98.0   
4  1491898404543717797               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds  minimum_nights  maximum_nights  minimum_minimum_nights  \
0             1   1.0              32            1125                      32   
1            12   3.0              32             365                      32   
2             6   3.0               2              45                       2   
3             2   1.0               2             180                       2   
4             6   3.0               2             365                       2   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                    1125                    1125   
1                      32                     365                     365   
2                       2                      45                      45   
3                       2                     180                     180   
4                       2                    1125                    1125   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm  has_availability  \
0                    32.0                  1125.0                 1   
1                    32.0                   365.0                 1   
2                     2.0                    45.0                 1   
3                     2.0                   180.0                 1   
4                     2.0                  1125.0                 1   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1               25               49               66               135   
2                0                0                0                 0   
3               23               53               83               355   
4                0                0                0                 0   

   number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  \
0                 18                      0                       0   
1                  0                      0                       0   
2                 14                     14                       1   
3                 13                     13                       3   
4                 64                     42                       3   

   review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
0              4.940000                5.000000                   4.890000   
1              4.760427                4.794096                   4.748194   
2              4.870000                5.000000                   5.000000   
3              4.080000                4.080000                   4.000000   
4              4.800000                4.740000                   4.750000   

   review_scores_chec

In [9]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [10]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

def sanitize_column_names(df):
    df.columns = df.columns.str.replace('[', '', regex=True)\
                           .str.replace(']', '', regex=True)\
                           .str.replace('<', '', regex=True)\
                           .str.replace('>', '', regex=True)\
                           .str.replace(' ', '_', regex=True)\
                           .str.replace('[^A-Za-z0-9_]+', '', regex=True)
    return df

x_train = sanitize_column_names(x_train)
x_test = sanitize_column_names(x_test)

In [11]:
pos = y_train.value_counts()[0] / y_train.value_counts()[1]

### Predictor Selection

In [3]:
# Used all Predictors

### Model Tuning and Training

In [12]:
base_model = DecisionTreeClassifier(random_state=1) 

bm1 = AdaBoostClassifier(estimator=base_model, random_state=1, learning_rate=0.5, n_estimators=450) 
bm2 = CatBoostClassifier(random_state=1, learning_rate=0.07, max_depth=6, n_estimators=10000, reg_lambda=1, subsample=0.65)
bm3 = LGBMClassifier(random_state=1, learning_rate=0.05, max_depth=8, n_estimators=3000, reg_lambda=0.02, subsample=0.5)

meta_classifier = LogisticRegression()

stacking_model = StackingClassifier(
    estimators=[('A', bm1), ('C', bm2), ('L', bm3)],
    final_estimator=meta_classifier
)

stacking_model.fit(x_train, y_train)

y_pred = stacking_model.predict(x_test)

C:\Users\benja\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0:	learn: 0.6482793	total: 139ms	remaining: 23m 7s
1:	learn: 0.6139484	total: 146ms	remaining: 12m 8s
2:	learn: 0.5900160	total: 152ms	remaining: 8m 26s
3:	learn: 0.5667947	total: 157ms	remaining: 6m 31s
4:	learn: 0.5490996	total: 163ms	remaining: 5m 25s
5:	learn: 0.5341189	total: 168ms	remaining: 4m 39s
6:	learn: 0.5205384	total: 172ms	remaining: 4m 6s
7:	learn: 0.5059421	total: 178ms	remaining: 3m 41s
8:	learn: 0.4956495	total: 182ms	remaining: 3m 22s
9:	learn: 0.4849076	total: 188ms	remaining: 3m 7s
10:	learn: 0.4690463	total: 192ms	remaining: 2m 54s
11:	learn: 0.4612993	total: 198ms	remaining: 2m 44s
12:	learn: 0.4525683	total: 203ms	remaining: 2m 35s
13:	learn: 0.4454756	total: 208ms	remaining: 2m 28s
14:	learn: 0.4375850	total: 213ms	remaining: 2m 21s
15:	learn: 0.4309934	total: 217ms	remaining: 2m 15s
16:	learn: 0.4222568	total: 222ms	remaining: 2m 10s
17:	learn: 0.4167032	total: 227ms	remaining: 2m 5s
18:	learn: 0.4125161	total: 231ms	remaining: 2m 1s
19:	learn: 0.4091813	total

198:	learn: 0.1897051	total: 1.19s	remaining: 58.4s
199:	learn: 0.1895611	total: 1.19s	remaining: 58.4s
200:	learn: 0.1887230	total: 1.2s	remaining: 58.4s
201:	learn: 0.1884000	total: 1.2s	remaining: 58.4s
202:	learn: 0.1876662	total: 1.21s	remaining: 58.5s
203:	learn: 0.1871179	total: 1.22s	remaining: 58.5s
204:	learn: 0.1867575	total: 1.23s	remaining: 58.8s
205:	learn: 0.1861860	total: 1.24s	remaining: 58.8s
206:	learn: 0.1854841	total: 1.24s	remaining: 58.8s
207:	learn: 0.1843757	total: 1.25s	remaining: 58.8s
208:	learn: 0.1838685	total: 1.25s	remaining: 58.7s
209:	learn: 0.1830537	total: 1.26s	remaining: 58.6s
210:	learn: 0.1825738	total: 1.26s	remaining: 58.6s
211:	learn: 0.1824123	total: 1.27s	remaining: 58.5s
212:	learn: 0.1814149	total: 1.27s	remaining: 58.5s
213:	learn: 0.1813372	total: 1.28s	remaining: 58.5s
214:	learn: 0.1807593	total: 1.28s	remaining: 58.4s
215:	learn: 0.1798772	total: 1.29s	remaining: 58.4s
216:	learn: 0.1792868	total: 1.29s	remaining: 58.3s
217:	learn: 0.

359:	learn: 0.1230590	total: 1.99s	remaining: 53.3s
360:	learn: 0.1230292	total: 1.99s	remaining: 53.3s
361:	learn: 0.1229329	total: 2s	remaining: 53.2s
362:	learn: 0.1226802	total: 2s	remaining: 53.2s
363:	learn: 0.1222418	total: 2.01s	remaining: 53.2s
364:	learn: 0.1220067	total: 2.01s	remaining: 53.1s
365:	learn: 0.1216724	total: 2.02s	remaining: 53.1s
366:	learn: 0.1212219	total: 2.02s	remaining: 53.1s
367:	learn: 0.1209640	total: 2.03s	remaining: 53.1s
368:	learn: 0.1206499	total: 2.03s	remaining: 53s
369:	learn: 0.1203413	total: 2.04s	remaining: 53s
370:	learn: 0.1201335	total: 2.04s	remaining: 53s
371:	learn: 0.1198876	total: 2.04s	remaining: 52.9s
372:	learn: 0.1198549	total: 2.05s	remaining: 52.9s
373:	learn: 0.1194106	total: 2.05s	remaining: 52.9s
374:	learn: 0.1190689	total: 2.06s	remaining: 52.9s
375:	learn: 0.1186509	total: 2.06s	remaining: 52.8s
376:	learn: 0.1182625	total: 2.07s	remaining: 52.8s
377:	learn: 0.1180165	total: 2.07s	remaining: 52.8s
378:	learn: 0.1177945	to

542:	learn: 0.0818542	total: 2.8s	remaining: 48.8s
543:	learn: 0.0817550	total: 2.81s	remaining: 48.8s
544:	learn: 0.0816043	total: 2.81s	remaining: 48.8s
545:	learn: 0.0813524	total: 2.82s	remaining: 48.8s
546:	learn: 0.0812266	total: 2.82s	remaining: 48.8s
547:	learn: 0.0809759	total: 2.83s	remaining: 48.8s
548:	learn: 0.0807391	total: 2.83s	remaining: 48.7s
549:	learn: 0.0805190	total: 2.84s	remaining: 48.7s
550:	learn: 0.0803844	total: 2.84s	remaining: 48.7s
551:	learn: 0.0800662	total: 2.84s	remaining: 48.7s
552:	learn: 0.0798972	total: 2.85s	remaining: 48.7s
553:	learn: 0.0797590	total: 2.85s	remaining: 48.6s
554:	learn: 0.0796912	total: 2.86s	remaining: 48.6s
555:	learn: 0.0794442	total: 2.86s	remaining: 48.6s
556:	learn: 0.0794299	total: 2.87s	remaining: 48.6s
557:	learn: 0.0792723	total: 2.87s	remaining: 48.6s
558:	learn: 0.0791101	total: 2.87s	remaining: 48.5s
559:	learn: 0.0788190	total: 2.88s	remaining: 48.5s
560:	learn: 0.0786099	total: 2.88s	remaining: 48.5s
561:	learn: 0

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

### Prediction

In [13]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('ensemble_Classifier_submission.csv', index=False)
results.head(30)

id  predicted
0   1543972437713169913          0
1   1710552057351883447          0
2              97075525          1
3              83734823          1
4              56722823          0
5             103823601          1
6   1159028243437550345          0
7              92485765          0
8              11142401          1
9   1189875082289533291          0
10  1544820103115167001          1
11             26331127          0
12             58383089          1
13             64762469          0
14            100902043          1
15            103130147          0
16             94352581          1
17  1849015146225985937          0
18             91307779          1
19             53591385          0
20             84071033          0
21             99484949          0
22             97864023          0
23             31809543          0
24  1519081708190436577          1
25             59771473          1
26  1310888620900225001          0
27             23054201          1
28             37470529          0
29  1212015341587401973          1